# Tutorial: Creating a Custom Task

TODO: words

In [2]:
import flash

import torch
from torch import nn

Creating a model. TODO: words

In [6]:
class LinearRegression(flash.Task):
    def __init__(self, num_inputs, learning_rate=0.1, metrics=None):
        # what kind of model do we want?
        model = nn.Linear(num_inputs, 1)

        # what loss function do we want?
        loss_fn = torch.nn.functional.mse_loss
        
        # what optimizer to do we want?
        optimizer = torch.optim.SGD
        
        super().__init__(
            model=model,
            loss_fn=loss_fn,
            optimizer=optimizer,
            metrics=metrics,
            learning_rate=learning_rate,
        )
        
    def forward(self, x):
        return self.model(x)